# Decorator Application

## Log and timing

In [1]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print(f'{run_dt}: called {fn.__name__}')
        return result
    
    return inner

In [2]:
@logged
def func1():
    x = 10
    pass

In [22]:
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner

In [23]:
@logged
@timed
def fact(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1,n+1))

In [24]:
fact(5)

fact ran for 0.000020s
2021-11-08 10:00:03.507532+00:00: called fact


120

In [25]:
fact(10)

fact ran for 0.000023s
2021-11-08 10:00:03.680422+00:00: called fact


3628800

### -----------------------------------------------------------------

## Caching

In [26]:
def caching(fn):

    cache = {}
    def inner(*args, **kwargs):
        args_key = str(args) + str(kwargs)
        if (args_key not in cache):
            #print("Calculate the first time!!")
            result = fn(*args, **kwargs)
            cache[args_key] = result
        #else:
            #print("Got you!!!")
        return cache[args_key]
    
    return inner

In [27]:
# @logged
@timed
@caching
def fibo(n):
    #print("running the function")
    
    if (n < 0):
        result = -1
    elif (n < 2):
        result = n
    else:
        # will re-caculate fibo(n-1) and fibo(n-2) so many times without caching
        result = fibo(n-1) + fibo(n-2)
    return result

In [28]:
fibo(100)

inner ran for 0.000005s
inner ran for 0.000013s
inner ran for 0.000365s
inner ran for 0.000016s
inner ran for 0.000728s
inner ran for 0.000011s
inner ran for 0.000888s
inner ran for 0.000005s
inner ran for 0.000987s
inner ran for 0.000004s
inner ran for 0.001078s
inner ran for 0.000004s
inner ran for 0.001170s
inner ran for 0.000004s
inner ran for 0.001262s
inner ran for 0.000004s
inner ran for 0.002495s
inner ran for 0.000014s
inner ran for 0.002862s
inner ran for 0.000005s
inner ran for 0.002933s
inner ran for 0.000005s
inner ran for 0.002997s
inner ran for 0.000004s
inner ran for 0.003062s
inner ran for 0.000004s
inner ran for 0.003126s
inner ran for 0.000003s
inner ran for 0.003360s
inner ran for 0.000005s
inner ran for 0.003428s
inner ran for 0.000004s
inner ran for 0.003497s
inner ran for 0.000004s
inner ran for 0.003560s
inner ran for 0.000004s
inner ran for 0.003624s
inner ran for 0.000003s
inner ran for 0.003689s
inner ran for 0.000004s
inner ran for 0.003755s
inner ran for 0.

354224848179261915075

In [29]:
fibo(100)

inner ran for 0.000012s


354224848179261915075

In [30]:
fibo(101)

inner ran for 0.000002s
inner ran for 0.000007s
inner ran for 0.002182s


573147844013817084101

In [31]:
class Fib:
    def __init__(self):
        self.cache = {0:0, 1:1}
        
    def fib(self, n):
        if n not in self.cache:
            print(f'calculating fib({n})')
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]

In [32]:
f = Fib()
f.fib(10)

calculating fib(10)
calculating fib(9)
calculating fib(8)
calculating fib(7)
calculating fib(6)
calculating fib(5)
calculating fib(4)
calculating fib(3)
calculating fib(2)


55

In [33]:
f.fib(10)

55

## -----------------------------------------------------------------

## Decorator with arguments

In [34]:
def timed_with_repitition(n):
    from functools import wraps
    from time import perf_counter
    
    def timed(fn):
        @wraps(fn)
        def inner(*args, **kwargs):
            start = perf_counter()
            for x in range(n):
                result = fn(*args, **kwargs)
            end = perf_counter()
            print('{0} ran for {1:.6f}s with average of {2:.6f}'.format(fn.__name__, end-start, (end-start)/n))
            return result
        return inner
    return timed

In [35]:
@timed_with_repitition(10000)
def fact1(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1,n+1))

In [36]:
fact1(1000)

fact1 ran for 1.758128s with average of 0.000176


4023872600770937735437024339230039857193748642107146325437999104299385123986290205920442084869694048004799886101971960586316668729948085589013238296699445909974245040870737599188236277271887325197795059509952761208749754624970436014182780946464962910563938874378864873371191810458257836478499770124766328898359557354325131853239584630755574091142624174743493475534286465766116677973966688202912073791438537195882498081268678383745597317461360853795345242215865932019280908782973084313928444032812315586110369768013573042161687476096758713483120254785893207671691324484262361314125087802080002616831510273418279777047846358681701643650241536913982812648102130927612448963599287051149649754199093422215668325720808213331861168115536158365469840467089756029009505376164758477284218896796462449451607653534081989013854424879849599533191017233555566021394503997362807501378376153071277619268490343526252000158885351473316117021039681759215109077880193931781141945452572238655414610628921879602238389714760

In [37]:
@timed_with_repitition(10000)
@caching
def fact1(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1,n+1))

In [38]:
fact1(1000)

inner ran for 0.008178s with average of 0.000001


4023872600770937735437024339230039857193748642107146325437999104299385123986290205920442084869694048004799886101971960586316668729948085589013238296699445909974245040870737599188236277271887325197795059509952761208749754624970436014182780946464962910563938874378864873371191810458257836478499770124766328898359557354325131853239584630755574091142624174743493475534286465766116677973966688202912073791438537195882498081268678383745597317461360853795345242215865932019280908782973084313928444032812315586110369768013573042161687476096758713483120254785893207671691324484262361314125087802080002616831510273418279777047846358681701643650241536913982812648102130927612448963599287051149649754199093422215668325720808213331861168115536158365469840467089756029009505376164758477284218896796462449451607653534081989013854424879849599533191017233555566021394503997362807501378376153071277619268490343526252000158885351473316117021039681759215109077880193931781141945452572238655414610628921879602238389714760